In [ ]:
import colorsys
import pickle
import random
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors as mcolors

random.seed(int("RND2022".replace("RND", "")))
# -----------------------------------------------------------------------------/


def gen_unique_random_color_pool(n_labels:list,
                                 existing_color_pool:dict=None,
                                 hsv_vthreshold:int=None) -> dict:
    """_summary_

    Args:
        n_labels (list): How many colors should generate
        existing_color_pool (dict, optional): If given, will extend the given color pool. Defaults to None.
        hsv_vthreshold (int, optional): Only collect the colors over this threshold. Defaults to None.

    Returns:
        dict: {Hex rgb: float rgb}
    """
    if existing_color_pool is None:
        color_pool = dict() # set 遇到重複的 value 不會 update
    else:
        color_pool = existing_color_pool
    
    color_pool["#000000"] = mcolors.hex2color("#000000")
    
    while len(color_pool) <= len(n_labels):
        
        # Generating a random number in between 0 and 2^24
        color = random.randrange(0, 2**24)
        hex_rgb = f"#{hex(color)[2:]:>06}" # `hex(color)` starts with `0x`
        float_rgb = mcolors.hex2color(hex_rgb) # range: [0.0, 1.0]
        
        if hsv_vthreshold is None:
            color_pool[f"{hex_rgb}"] = float_rgb # save color
        else:
            hsv_color = colorsys.rgb_to_hsv(*float_rgb)
            # 以 V channel (亮度) 進行判斷
            v_threshold = hsv_vthreshold/255 # 255 is `V_MAX` in OpenCV HSV color model
            if hsv_color[2] > v_threshold:
                color_pool[f"{hex_rgb}"] = float_rgb # save color
    
    color_pool.pop("#000000") # remove `black` (background)

    return color_pool
    # -------------------------------------------------------------------------/


# random color #1 (merge background)
seg1_pkl = Path(r"")

# random color #2 (merge same color cell)
seg2_pkl = Path(r"")

with open(seg1_pkl, 'rb') as file:
    seg1 = pickle.load(file)
unique_labels = np.unique(seg1)[1:] # 找出所有唯一的 label

# 依照 label 的數量生成不重複的顏色
color_pool = gen_unique_random_color_pool(unique_labels)
assert len(unique_labels) == len(color_pool)

# 創建與 segmentation 相同大小的 RGB 圖像
colored_seg1 = np.zeros((*seg1.shape, 3))  # 3 表示 RGB 三個通道

# 將每個 label 對應的顏色填入圖像
label2hexrgb: dict[str, str] = dict()
for label, (hex_rgb, float_rgb) in zip(unique_labels, color_pool.items()):
    assert label != 0
    colored_seg1[seg1 == label] = float_rgb
    label2hexrgb[label] = hex_rgb

# # 顯示結果
plt.imshow(colored_seg1)
plt.title('Label to RGB Segmentation with Unique Colors')
plt.axis('off')
plt.show()

with open(seg2_pkl, 'rb') as file:
    seg2 = pickle.load(file)
unique_labels = np.unique(seg2)[1:] # 找出所有唯一的 label

colored_seg2 = np.zeros((*seg2.shape, 3)) # 3 表示 RGB 三個通道

for label in unique_labels:
    colored_seg2[seg2 == label] = color_pool[label2hexrgb[label]]

# # 顯示結果
plt.imshow(colored_seg2)
plt.title('Label to RGB Segmentation with Unique Colors')
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt

def gen_singlecolor_palmskin(seg:np.ndarray,
                             cytosol_color: tuple[float, float, float]=(0.5, 0.5, 0.5),
                             border_color: tuple[float, float, float]=(1.0, 1.0, 1.0),
                             bg_color: tuple[float, float, float]=None,
                             ) -> np.ndarray:
    """
    """
    fake_palmskin = np.full((*seg.shape, 3), cytosol_color, dtype=np.float64)
    
    if bg_color is not None:
        fake_palmskin[seg == 0] = bg_color
    
    fake_palmskin = mark_boundaries(fake_palmskin, seg, color=border_color)

    return fake_palmskin
    # -------------------------------------------------------------------------/

img = gen_singlecolor_palmskin(seg1,
                               cytosol_color=0.5,
                               border_color=0.5,
                               bg_color=0.0)
plt.imshow(img)

In [ ]:
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from skimage import color

# random color #1 (merge background)
seg1_pkl = Path(r"")

# random color #2 (merge same color cell)
seg2_pkl = Path(r"")

with open(seg1_pkl, 'rb') as file:
    segmentation = pickle.load(file)
unique_labels = np.unique(segmentation)

rgb_image = color.label2rgb(
    segmentation, colors=np.random.random((len(unique_labels), 3)),
    bg_label=0
)

# 顯示結果
plt.imshow(rgb_image)
plt.title('Label to RGB Segmentation with Unique Colors')
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
from collections import Counter

# np.random.seed(2022)

# colors = np.random.random((300, 3))
colors = gen_unique_random_color_pool(range(30000))

color_pool_hex = dict()
Counter([mcolors.rgb2hex(color) for color in colors]).most_common(5)